# 5 - Model evaluation

## Our model certainly out-perform baseline model

In [2]:
import numpy as np
import pandas as pd

from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.optimizers import *
from keras.losses import *
from keras import backend as K

import tensorflow as tf
print(tf.__version__)

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

Using TensorFlow backend.


1.14.1-dev20190607


In [18]:
baseline_metrics = pd.read_csv('../metrics/baseline_using_prev_datapoint.csv')
model_metrics = pd.read_csv('../metrics/stacked_model.csv')

In [19]:
baseline_metrics.describe()

,rmse(t+1),rmse(t+2),rmse(t+3),rmse(t+4),rmse(t+5),smape(t+1),smape(t+2),smape(t+3),smape(t+4),smape(t+5)
count,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000
mean,0.013834,0.016883,0.018968,0.020827,0.022719,42.098473,47.293172,50.143613,52.520402,54.797069
std,0.012506,0.015951,0.018837,0.021662,0.024632,22.140788,24.159773,25.002974,25.662177,26.237707
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.003495,0.004101,0.004290,0.004392,0.004505,25.949403,30.750037,32.944610,35.290293,37.799933
50%,0.011329,0.013164,0.014004,0.014685,0.015370,44.076585,50.214083,54.137186,57.647730,61.374586
75%,0.020779,0.025035,0.027513,0.029919,0.032117,59.777084,66.124484,69.072297,71.632403,74.284731
max,0.067160,0.086754,0.106129,0.124682,0.144874,97.764010,105.237471,105.171687,110.197537,113.192732


In [20]:
model_metrics.describe()

,rmse(t+1),rmse(t+2),rmse(t+3),rmse(t+4),rmse(t+5),smape(t+1),smape(t+2),smape(t+3),smape(t+4),smape(t+5)
count,1329.000000,1.329000e+03,1329.000000,1.329000e+03,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000
mean,0.012436,1.376487e-02,0.014461,1.498439e-02,0.015406,51.652021,50.591721,50.100544,51.259864,50.680409
std,0.011131,1.274595e-02,0.013718,1.444920e-02,0.015114,26.546016,26.096379,26.335638,25.571516,25.539954
min,0.000000,1.047339e-07,0.000000,5.841255e-08,0.000000,0.000000,0.190476,0.000000,0.380952,0.000000
25%,0.003463,3.632281e-03,0.003660,3.785942e-03,0.003778,32.204681,31.699774,31.008934,32.578618,32.313670
50%,0.010054,1.071960e-02,0.011083,1.131061e-02,0.011445,51.602022,50.091757,50.462803,51.784532,51.727902
75%,0.018477,2.030096e-02,0.021047,2.158792e-02,0.022023,71.597790,69.367944,69.121369,70.181673,69.760072
max,0.062792,8.009573e-02,0.092854,1.016356e-01,0.111073,120.230463,121.409065,114.904273,110.065438,109.557381


# Evaluation/Submission
- Assume similar format as training.csv
- The cells below extract exactly from other notebooks
- Run a similar data preprocessing and feature engineering
- predict next 5 demands

###### This might get messy as my pipeline will get 7 days gap instead of 14 days gap

### Features
- Distance/datetime features
  - distance_from_qp09d8, distance_from_qp03xx, distance_from_qp03wf, day_of_week_sin, day_of_week_cos, hour_sin, hour_cos, fifteen_minute_sin, fifteen_minute_cos
- Last 2 hours demands
  - demand_t0, demand_t1, demand_t2, demand_t3, demand_t4, demand_t5, demand_t6, demand_t7
- Last 7th day demands
  - demand_7d_t{i-th} for i in range(13)
- last 6 days demands
  - demand_{day}d_t{i-th} for day in range(1,7), for i in range(13)

In [22]:
def root_mean_squared_error(y_true, y_pred):
    '''RMSE in keras'''
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def symmetric_mean_absolute_mean_error(y_true, y_pred):
    '''SMAPE - modified (got it from kaggle)'''
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

def calculate_rmse(x, y):
    '''RMSE in numpy'''
    return np.sum(np.absolute(x - y))/x.shape[0]

def calculate_smape(y_true, y_pred):
    '''This is just a function wrapper lol'''
    return symmetric_mean_absolute_mean_error(y_true, y_pred)

In [25]:
model = load_model('../model/final_stacked_100.h5', 
                   custom_objects={'root_mean_squared_error': root_mean_squared_error})

W0617 08:08:14.335798 4579440064 deprecation.py:323] From /anaconda3/envs/grab/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1251: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
# assuming that this file has similar format as training.csv
# assuming that this file has 14 days of demands - predict the next 5
test_df = pd.read_csv('../data/test.csv')

In [6]:
# 1. fill missing values (=0)
def get_timestamp_order(ts):
    '''timestamp to order'''
    ts = ts.split(':')
    return int(ts[0])*4 + int(ts[1])//15

def timestamp_to_hour_minute(timestamp):
    '''timestamp to hour, minute'''
    timestamp = timestamp.split(':')
    return int(timestamp[0]), int(timestamp[1])

def format_order(row):
    '''get order'''
    return (row['day'] - 1)*96 + get_timestamp_order(row['timestamp']) # day is 1-indexed

def order_to_time(order):
    '''order to hour, minute'''
    order = order % 96
    return order//4, order%4 * 15

test_df['order'] = test_df.apply(format_order, axis=1)

formatted_dict = {}
GEOHASH_LIST = test_df.geohash6.unique()
MIN_ORDER = np.min(test_df.order.unique())
MAX_ORDER = np.max(test_df.order.unique())
for i, gh in enumerate(GEOHASH_LIST):
    temp_df = test_df[test_df.geohash6 == gh]
    temp_df.set_index('order', inplace=True)
    temp_dict = temp_df.to_dict('index')
    for order in range(MIN_ORDER, MAX_ORDER+1):
        hour, minute = order_to_time(order)
        day = order // 96
        if order not in temp_dict:
            demand = 0.0
        else:
            demand = temp_dict[order]['demand']
        formatted_dict[order + i*MAX_ORDER] = {
            'order': order,
            'geohash6': gh,
            'timestamp': '{0}:{1}'.format(hour, minute),
            'day': day,
            'demand': demand,
        }
        
test_df = pd.DataFrame.from_dict(formatted_dict, orient='index')
# test_df.to_pickle('../data/0-test-df.pkl')

In [8]:
# 2. data-preproc/feat-eng
import geohash
import gc
from geopy import distance

def timestamp_to_hour_minute(timestamp):
    timestamp = timestamp.split(':')
    return int(timestamp[0]), int(timestamp[1])

def get_distance_by_gh(gh1, gh2):
    coor1 = geohash.decode(gh1)
    coor2 = geohash.decode(gh2)
    return distance.distance(coor1, coor2).km

test_hour = test_df.timestamp.apply(lambda x: timestamp_to_hour_minute(x)[0])
test_df['hour_sin'] = np.sin(test_hour*(np.pi/24))
test_df['hour_cos'] = np.cos(test_hour*(np.pi/24))

test_day = test_df.day.values % 7
test_df['day_of_week_sin'] = np.sin(test_day*(np.pi/7))
test_df['day_of_week_cos'] = np.cos(test_day*(np.pi/7))

test_fifteen_min = test_df.order.values % 96
test_df['fifteen_minute_sin'] = np.sin(test_fifteen_min*(np.pi/96))
test_df['fifteen_minute_cos'] = np.cos(test_fifteen_min*(np.pi/96))

test_df['lat'] = test_df.geohash6.apply(lambda x: geohash.decode(x)[0])
test_df['long'] = test_df.geohash6.apply(lambda x: geohash.decode(x)[1])

POI_GEOHASH = ['qp09d8', 'qp03xx', 'qp03wf']
poi_distance_dict = {}
for poi in POI_GEOHASH:
    poi_distance_dict[poi] = {}
    for gh in test_df.geohash6.unique():
        poi_distance_dict[poi][gh] = get_distance_by_gh(poi, gh)
    
for gh in POI_GEOHASH:
    test_df['distance_from_{0}'.format(gh)] = test_df.geohash6.apply(lambda x: poi_distance_dict[gh][x])
    
for gh in POI_GEOHASH:
    gh_key = 'distance_from_{0}'.format(gh)
    gh_mean = 17.842158336193567 # train_df[gh_key].mean()
    gh_std = 9.672254418484291 # train_df[gh_key].std()
    test_df[gh_key] = (test_df[gh_key] - gh_mean)/gh_std

    
csv_file = open('../data/test_df.csv', 'w')

headers = [
    'order', 'geohash6', 'day', 'timestamp',
    'distance_from_qp09d8', 'distance_from_qp03xx', 'distance_from_qp03wf',
    'demand_t0', 'demand_t1', 'demand_t2', 'demand_t3', 'demand_t4', 'demand_t5', 'demand_t6', 'demand_t7',
    'day_of_week_sin', 'day_of_week_cos',
    'hour_sin', 'hour_cos',
    'fifteen_minute_sin', 'fifteen_minute_cos',
]

for d in range(1, 8):
    for i in range(13):
        headers.append('demand_{0}d_t{1}'.format(d, i))

csv_file.write(','.join(headers))
csv_file.write('\n')

for i, gh in enumerate(test_df.geohash6.unique()):
    # print(i, len(df.geohash6.unique()))
    if i % 100 == 0: print(i)
    temp_df = test_df[test_df.geohash6 == gh]
    temp_df.set_index('order', inplace=True)
    start_row = temp_df.index[0]
    end_rows = temp_df.index[-1]
    for i in range(start_row + 96*7 + 3, end_rows):
        # put in whatever data needed into here
        arr = {
            'order': i,
            'geohash6': gh,
            'day': temp_df.at[i, 'day'],
            'timestamp': temp_df.at[i, 'timestamp'],
            
            'distance_from_qp09d8': temp_df.at[i, 'distance_from_qp09d8'],
            'distance_from_qp03xx': temp_df.at[i, 'distance_from_qp03xx'],
            'distance_from_qp03wf': temp_df.at[i, 'distance_from_qp03wf'],
            
            # Last two hours demands
            'demand_t0': temp_df.at[i, 'demand'],
            'demand_t1': temp_df.at[i-1, 'demand'],
            'demand_t2': temp_df.at[i-2, 'demand'],
            'demand_t3': temp_df.at[i-3, 'demand'],
            'demand_t4': temp_df.at[i-4, 'demand'],
            'demand_t5': temp_df.at[i-5, 'demand'],
            'demand_t6': temp_df.at[i-6, 'demand'],
            'demand_t7': temp_df.at[i-7, 'demand'],
            
            'day_of_week_sin': temp_df.at[i, 'day_of_week_sin'],
            'day_of_week_cos': temp_df.at[i, 'day_of_week_cos'],
            
            'hour_sin': temp_df.at[i, 'hour_sin'],
            'hour_cos': temp_df.at[i, 'hour_cos'],
            
            'fifteen_minute_sin': temp_df.at[i, 'fifteen_minute_sin'],
            'fifteen_minute_cos': temp_df.at[i, 'fifteen_minute_cos'],
        }
        
        for d in range(1, 8):
            for j in range(13):
                arr['demand_{0}d_t{1}'.format(d, j)] = temp_df.at[i + j - 96*d - 3, 'demand']
        
        content = []
        for h in headers:
            content.append(str(arr[h]))
        csv_file.write(','.join(content))
        csv_file.write('\n')
    gc.collect()
    
csv_file.close()

0


In [9]:
test_df = pd.read_csv('../data/test_df.csv')

In [10]:
test_df.head()

,order,geohash6,day,timestamp,distance_from_qp09d8,distance_from_qp03xx,distance_from_qp03wf,demand_t0,demand_t1,demand_t2,...,demand_7d_t3,demand_7d_t4,demand_7d_t5,demand_7d_t6,demand_7d_t7,demand_7d_t8,demand_7d_t9,demand_7d_t10,demand_7d_t11,demand_7d_t12
0,5475,qp03wc,57,0:45,0.044795,-1.311147,-1.78187,0.239678,0.204246,0.186009,...,0.206627,0.232386,0.255655,0.247960,0.275323,0.238671,0.179991,0.191984,0.243441,0.269081
1,5476,qp03wc,57,1:0,0.044795,-1.311147,-1.78187,0.245050,0.239678,0.204246,...,0.232386,0.255655,0.247960,0.275323,0.238671,0.179991,0.191984,0.243441,0.269081,0.207029
2,5477,qp03wc,57,1:15,0.044795,-1.311147,-1.78187,0.238839,0.245050,0.239678,...,0.255655,0.247960,0.275323,0.238671,0.179991,0.191984,0.243441,0.269081,0.207029,0.245731
3,5478,qp03wc,57,1:30,0.044795,-1.311147,-1.78187,0.175717,0.238839,0.245050,...,0.247960,0.275323,0.238671,0.179991,0.191984,0.243441,0.269081,0.207029,0.245731,0.256975
4,5479,qp03wc,57,1:45,0.044795,-1.311147,-1.78187,0.192860,0.175717,0.238839,...,0.275323,0.238671,0.179991,0.191984,0.243441,0.269081,0.207029,0.245731,0.256975,0.282395


In [13]:
# 3. filter the to-be predicted datetime given T (day, timestamp) of that geohash6
to_predict = test_df[(test_df.day == 57) & (test_df.timestamp == '1:0') & (test_df.geohash6 == 'qp03wc')]

In [26]:
# 4. predict
raw_feat = [
    'distance_from_qp09d8', 'distance_from_qp03xx', 'distance_from_qp03wf',
    'day_of_week_sin', 'day_of_week_cos',
    'hour_sin', 'hour_cos',
    'fifteen_minute_sin', 'fifteen_minute_cos',
]

last_two_hour_feat = ['demand_t{0}'.format(t) for t in range(8)]

last_six_day_feat = []
for d in range(1,7):
    for i in range(13):
        last_six_day_feat.append('demand_{0}d_t{1}'.format(d, i))

last_week_today_feat = ['demand_7d_t{0}'.format(i) for i in range(13)]

X_test_raw = to_predict[raw_feat]
X_test_last_two_hour = to_predict[last_two_hour_feat]
X_test_last_week = to_predict[last_week_today_feat]
X_test_last_six_day = to_predict[last_six_day_feat]

In [29]:
preds = model.predict([
    X_test_raw,
    X_test_last_week,
    X_test_last_six_day,
    X_test_last_two_hour,
])

preds

array([[0.2274777 , 0.21903622, 0.21379256, 0.21304029, 0.21263045]],
      dtype=float32)